# Algorytmy tekstowe 2019/2020

# Laboratorium 5

# Autor - Łukasz Jezapkowicz

# Zadanie dotyczy wyszukiwania wzorców dwuwymiarowych.

# 0. Przygotowanie danych

Dla tak małego pliku jakim jest "haystack.txt" uznałem, że nie będzie mocno krzywdzące (pod względem wydajności) wypełnienie go wybranym symbolem '#' w celu osiągnięcia tekstu o równej długości wszystkich linii.

In [3]:
# otwieranie pliku
def open_file(file):
    file = open(file,mode='r', encoding="utf-8")
    data = file.read()
    file.close()
    return data

In [4]:
# wypełnianie pliku znakiem '#' (nie występuje ani razu) w celu równej długości każdej linii
def adapt_text(text):
    lines = text.splitlines()
    new_lines = []
    max_number_of_lines = 0
    for line in lines:
        if len(line) > max_number_of_lines:
            max_number_of_lines = len(line)
    
    for line in lines:
        new_lines.append(list(line.ljust(max_number_of_lines,'#')))
    
    return new_lines

Tekst przechowuje jako liste list w celu lepszego dostępu do jej elementów w algorytmie.

In [5]:
haystack_text = adapt_text(open_file("haystack.txt"))
print(str(haystack_text[0][:20]) + '...')
print(str(haystack_text[1][:20]) + '...')
print(str(haystack_text[2][:20]) + '...')
print('...')

['O', 'n', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 's', 'i', 'm', 'p', 'l', 'e', 's', 't', ' ']...
['o', 'f', ' ', 'w', 'r', 'i', 't', 't', 'e', 'n', ' ', 't', 'e', 'x', 't', 's', '.', ' ', 'T', 'h']...
['b', 'e', ' ', 'w', 'r', 'i', 't', 't', 'e', 'n', ' ', 'd', 'o', 'w', 'n', ' ', 'a', 's', ' ', 'a']...
...


Obraz "haystack.png" przechowywać będę jako macierz dwuwymiarową, w której każdy element zawiera albo 0 albo 1 (tryb czarno-biały)

In [6]:
# funkcja zwracająca zdjęcie jako tablicę dwuwymiarową o wartościach 0/1
def convert_to_binary_array(image):
    new_image = image.convert('P', palette=Image.ADAPTIVE, colors=256)
    A = array(new_image)
    data = []
    
    for i in range(len(A)):
        column = []
        for j in range(len(A[i])):
            column.append(A[i][j])
        data.append(column)
    return data

In [7]:
from PIL import Image
from numpy import empty,array

haystack_image = Image.open('haystack.png')
haystack_image_data = convert_to_binary_array(haystack_image)
            
print(str(haystack_image_data[0][:20]) + '...')
print(str(haystack_image_data[1][:20]) + '...')
print(str(haystack_image_data[2][:20]) + '...')
print('...')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
...


# 1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego.

Na początku zaimplementuję funkcje, które budują automat dla podanej listy wzorców.

In [8]:
# klasa symbolizująca węzeł
class Node:
    def __init__(self, index):
        self.goto = {}
        self.out = []
        self.fail = None
        self.index = index

# funkcja tworząca nowy węzeł dla każdego znaku w każdym wzorcu
def create_forest(patterns):
    root = Node(0)
     
    i = 1
    for path in patterns:
        node = root
        for symbol in path:
            node = node.goto.setdefault(symbol, Node(i))
            i += 1
        node.out.append(path)
    return root

# funkcja budująca automat dla listy wzorców - Aho-Corasick
def build_automata(patterns):
    root = create_forest(patterns)
    queue = []
    for node in root.goto.values():
        queue.append(node)
        node.fail = root
 
    while len(queue) > 0:
        rnode = queue.pop(0)
 
        for key, unode in rnode.goto.items():
            queue.append(unode)
            fnode = rnode.fail
            while fnode != None and not key in fnode.goto:
                fnode = fnode.fail
            unode.fail = fnode.goto[key] if fnode else root
            unode.out += unode.fail.out
 
    return root

In [9]:
# Wyszukiwanie wzorca jednowymiarowego - z laboratorium 1
def KMPSearch(txt,pattern):
    S = set()
    K = []  
    t = -1
    K.append(t)
    for k in range(1, len(pattern) + 1):
        while(t >= 0 and pattern[t] != pattern[k - 1]):
            t = K[t]
        t = t + 1  
        K.append(t)
    m = 0  
    for i in range(0, len(txt)):
        while (m >= 0 and pattern[m] != txt[i]):
            m = K[m]
        m = m + 1  
        if m == len(pattern):
            S.add(i-m+1)
            m = K[m]
    return len(S), S

In [15]:
from time import time
# funkcja wyszukująca wzorców dwuwymiarowych, patterns to lista kolejnych kolumn wzorca 2d
def search_patterns(text,patterns,print_output=True):
    time1 = time()
    automata = build_automata(patterns)
    
    # numery kolejnych stanów prawidłowych
    patterns_pattern = []
    p_list = []
    current = 0
    for pattern in patterns:
        current += len(pattern)
        num = current
        for p in p_list:
            if p[0] == pattern:
                num = p[1]
                
        patterns_pattern.append(num)
        if num == current:
            p_list.append((pattern,num))
        
    states = [[0 for _ in range(len(text[0]))] for _ in range(len(text))]
        
    # wypełniam macierz stanów dla kolejnych kolumn
    for j in range(len(text[0])):
        # kolumna j
        node = automata
        for i in range(len(text)):
            while node != None and not text[i][j] in node.goto:
                node = node.fail
            if node == None:
                node = automata
                continue
            node = node.goto[text[i][j]]
            states[i][j] = node.index
    
    time2 = time()
    # dla każdego wiersza macierzy stanów poszukuje "patterns_pattern"
    i = -1 * len(patterns[0]) + 1
    count = 0
    for row in states:
        lenS, S = KMPSearch(row,patterns_pattern)
        count += lenS
        if print_output:
            for j in S:
                print("Pattern found: (" + str(i) + "," + str(j) + ") ... (" + str(i) + "," + str(j+len(patterns)-1) + ")")
                print("               ...")
                print("               (" + str(i+len(patterns[0])-1) + "," + str(j) + ") ... (" + \
                      str(i+len(patterns[0])-1) + "," + str(j+len(patterns)-1) + ")")
        i += 1
    print("Total number of pattern occurences = " + str(count))
    time3 = time()
    
    print("Automata building time => " + str(1000 * (time2-time1)) + " milliseconds")
    print("Searching time => " + str(1000 * (time3-time2)) + " milliseconds")
    return count

# 2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku.

Z problemem nierównej długości linii w pliku uporałem się już w podpunkcie 0. Nie będę więc sobie nim już więcej zawracał głowy.  
  
  
Żeby nie wykonywać algorytmu 256 razy, na początku w pojedynczym przejściu znajdę wszystkie litery, które występują w tym tekście.

In [11]:
letters = set()
for row in haystack_text:
    for letter in row:
        letters.add(letter)

Teraz dla każdej występującej literki $x$ wyszukam wzorca $[[x],[x]]$ czyli pojedynczej kolumny o dwóch wierszach i wartościach $x$.  
  
Ponieważ moja funkcja przyjmuje liste wzorców, które są kolejnymi kolumnami wzorca dwuwymiarowego to do funkcji przekażę liste o jednym elemencie $[xx]$

In [13]:
count = 0
for letter in letters:
    print("\nLetter '" + letter + "':")
    count += search_patterns(haystack_text,[letter+letter],False)


Letter 'd':
Total number of pattern occurences = 1
Automata building time => 1.9769668579101562 milliseconds
Searching time => 1.9946098327636719 milliseconds

Letter 'b':
Total number of pattern occurences = 0
Automata building time => 1.9948482513427734 milliseconds
Searching time => 1.995086669921875 milliseconds

Letter 'A':
Total number of pattern occurences = 0
Automata building time => 1.995086669921875 milliseconds
Searching time => 1.9946098327636719 milliseconds

Letter 'W':
Total number of pattern occurences = 0
Automata building time => 1.9941329956054688 milliseconds
Searching time => 2.992391586303711 milliseconds

Letter 'w':
Total number of pattern occurences = 2
Automata building time => 3.0193328857421875 milliseconds
Searching time => 1.9941329956054688 milliseconds

Letter '3':
Total number of pattern occurences = 0
Automata building time => 1.9683837890625 milliseconds
Searching time => 1.9946098327636719 milliseconds

Letter '5':
Total number of pattern occurence

In [14]:
print("In total, there were " + str(count) + " occurences of such situations in text")

In total, there were 2420 occurences of such situations in text


Jak widać, jest to dosyć częsta sytuacja.  
  
Po ręcznym sprawdzeniu kilku wartości stwierdzam, iż algorytm odszukuje poprawnie taki wzorzec.

# 3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji.

Zacznijmy od "th" w dwóch kolejnych liniach na tej samej pozycji.  
  
Jako liste wzorców podaje kolejne kolumny wzorca czyli $[tt,hh]$

In [16]:
count = search_patterns(haystack_text,['tt','hh'])

Total number of pattern occurences = 0
Automata building time => 2.02178955078125 milliseconds
Searching time => 1.9946098327636719 milliseconds


Ręczne przeszukanie pliku potwierdza, iż faktycznie nie ma takiej sytuacji w tym pliku.  
  
Teraz dla "t h" w dwóch kolejnych liniach na tej samej pozycji.  
  
Jako liste wzorców podaje kolejne kolumny wzorca czyli $[tt,--,hh]$, gdzie $-$ symbolizuje pusty znak.

In [17]:
count = search_patterns(haystack_text,['tt','  ','hh'])

Pattern found: (37,0) ... (37,2)
               ...
               (38,0) ... (38,2)
Total number of pattern occurences = 1
Automata building time => 2.992391586303711 milliseconds
Searching time => 1.9938945770263672 milliseconds


Ręczne przeszukanie pliku potwierdza, iż faktycznie taka sytuacja występuje tylko raz, w miejscu wskazanym przez algorytm.  
  
Stwierdzam zatem, iż algorytm działa poprawnie dla pliku tekstowego "haystack.txt".

# 4. Wybierz przynajmniej 4 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png".

Na początku manipulując metodą Image.crop postaram się wyciąć odpowiednie wzorce dla małych liter.  
  
Literki ścinałem tak długo jak długo program zwracał niepoprawny wynik.

In [18]:
from IPython.display import Image

In [19]:
# funkcja zwracajaca odpowiedni wycinek macierzy
def get_data(left,upper,right,lower):
    data = []
    for row in haystack_image_data[upper:lower]:
        data.append(row[left:right])
    return data

Litera e:

In [20]:
pattern1 = haystack_image.crop((591,565,600,575))
pattern1_data = get_data(591,565,600,575)
Image(filename='e.png')

Litera c:

In [21]:
pattern2 = haystack_image.crop((486,564,494,575))
pattern2_data = get_data(486,564,494,575)
Image(filename='c.png')

Litera o:

In [22]:
pattern3 = haystack_image.crop((214,564,223,576))
pattern3_data = get_data(214,564,223,576)
Image(filename='o.png')

Litera u:

In [23]:
pattern4 = haystack_image.crop((608,124,617,137))
pattern4_data = get_data(608,124,617,137)
Image(filename='u.png')

Litera x:

In [24]:
pattern5 = haystack_image.crop((366,231,376,247))
pattern5_data = get_data(366,231,376,247)
Image(filename='x.png')

Litera a:

In [25]:
pattern6 = haystack_image.crop((120,234,129,245))
pattern6_data = get_data(120,234,129,245)
Image(filename='a.png')

In [26]:
# funkcja zwracająca kolumny podanego wzorca
def get_patterns_columns(data):
    patterns = []
    for j in range(len(data[0])):
        column = []
        for i in range(len(data)):
            column.append(data[i][j])
        patterns.append(column)
    return patterns

In [27]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern1_data),False)

Total number of pattern occurences = 583
Automata building time => 906.6071510314941 milliseconds
Searching time => 387.92920112609863 milliseconds


In [28]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern2_data),False)

Total number of pattern occurences = 213
Automata building time => 847.722053527832 milliseconds
Searching time => 376.07860565185547 milliseconds


In [29]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern3_data),False)

Total number of pattern occurences = 369
Automata building time => 864.7134304046631 milliseconds
Searching time => 384.9446773529053 milliseconds


In [30]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern4_data),False)

Total number of pattern occurences = 106
Automata building time => 847.7692604064941 milliseconds
Searching time => 372.9665279388428 milliseconds


In [31]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern5_data),False)

Total number of pattern occurences = 57
Automata building time => 866.680383682251 milliseconds
Searching time => 382.01117515563965 milliseconds


In [32]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern6_data),False)

Total number of pattern occurences = 397
Automata building time => 877.6512145996094 milliseconds
Searching time => 364.0270233154297 milliseconds


Podsumowanie:  
  
Litera e:  Ilość wystąpień w tekście (Notepad) = 583, Wartość zwrócona przez program = 583  
  
Litera c:  Ilość wystąpień w tekście (Notepad) = 213, Wartość zwrócona przez program = 213  
  
Litera o:  Ilość wystąpień w tekście (Notepad) = 369, Wartość zwrócona przez program = 369  
  
Litera u:  Ilość wystąpień w tekście (Notepad) = 106, Wartość zwrócona przez program = 106  
  
Litera x:  Ilość wystąpień w tekście (Notepad) = 57, Wartość zwrócona przez program = 57  
  
Litera a:  Ilość wystąpień w tekście (Notepad) = 397, Wartość zwrócona przez program = 397  
  
Wszystko zatem działa zgodnie z założeniami.

# 5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png.

In [33]:
pattern7 = haystack_image.crop((422,494,530,515))
pattern7_data = get_data(422,494,530,515)
Image(filename='pattern.png')

In [34]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern7_data))

Pattern found: (472,178) ... (472,285)
               ...
               (492,178) ... (492,285)
Pattern found: (494,422) ... (494,529)
               ...
               (514,422) ... (514,529)
Pattern found: (538,236) ... (538,343)
               ...
               (558,236) ... (558,343)
Pattern found: (582,136) ... (582,243)
               ...
               (602,136) ... (602,243)
Pattern found: (626,242) ... (626,349)
               ...
               (646,242) ... (646,349)
Total number of pattern occurences = 5
Automata building time => 886.6548538208008 milliseconds
Searching time => 471.74644470214844 milliseconds


Również tutaj Notepad zwraca wartość identyczną z programem (5) co pokazuje jego poprawność.

# 6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca.

In [35]:
print(len(haystack_text))
pat1 = ['abc','bcd','dcd','123']
count = search_patterns(haystack_text,pat1)

84
Total number of pattern occurences = 0
Automata building time => 2.9916763305664062 milliseconds
Searching time => 1.9948482513427734 milliseconds


Jak widać czas budowania automatu jest mały dla takiego małego wzorca. Czas wyszukiwania również jest bardzo krótki.

In [36]:
pat2 = []
tmp = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
for i in range(100):
    pat2.append(tmp)
count = search_patterns(haystack_text,pat2)

Total number of pattern occurences = 0
Automata building time => 3.9887428283691406 milliseconds
Searching time => 4.97889518737793 milliseconds


Czas budowania automatu dla większego wzorca jest nieco większy lecz wciąż mały. Czas wyszukiwania zwiększa się, ponieważ  
wyszukiwany wzorzec jednowymiarowy jest dłuższy.

In [37]:
pat3 = []
for j in range(len(haystack_text[0])):
    column = ''
    for i in range(len(haystack_text)):
        column += haystack_text[i][j]
    pat3.append(column)
count = search_patterns(haystack_text,pat3)

Pattern found: (0,0) ... (0,104)
               ...
               (83,0) ... (83,104)
Total number of pattern occurences = 1
Automata building time => 22.91131019592285 milliseconds
Searching time => 3.9887428283691406 milliseconds


Czas budowania automatu dla wzorca, który jest całym tekstem jest zauważalnie dłuższy. Czas wyszukiwania nieco zwiększył się z tego samego powodu co powyżej.

In [38]:
pat4 = []
tmp = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
for i in range(1000):
    pat4.append(tmp)
count = search_patterns(haystack_text,pat4)

Total number of pattern occurences = 0
Automata building time => 24.933815002441406 milliseconds
Searching time => 21.97718620300293 milliseconds


Czas budowania automatu jest podobny co powyżej, lecz czas wyszukiwania jest dłuższy z powodu większej ilości kolumn.

In [39]:
pat5 = []
tmp = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
for i in range(100000):
    pat5.append(tmp)
count = search_patterns(haystack_text,pat5)

Total number of pattern occurences = 0
Automata building time => 1951.8132209777832 milliseconds
Searching time => 1833.0628871917725 milliseconds


Czas budowania automatu i wyszukiwania jest dłuższy niemal 100 razy (100 razy więcej kolumn) -> pokazuje to liniowość rozwiązania.

# 7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania.

W celu wykonania tego doświadczenia zmodyfikuję lekko funkcję search_patterns żeby "symulowała" wielowątkowość.

In [40]:
from math import ceil
# funkcja wyszukująca wzorców dwuwymiarowych, patterns to lista kolejnych kolumn wzorca 2d
def search_patterns_threads(text,patterns,threads):
    time1 = time()
    automata = build_automata(patterns)
    
    # numery kolejnych stanów prawidłowych
    patterns_pattern = []
    p_list = []
    current = 0
    for pattern in patterns:
        current += len(pattern)
        num = current
        for p in p_list:
            if p[0] == pattern:
                num = p[1]
                
        patterns_pattern.append(num)
        if num == current:
            p_list.append((pattern,num))
        
    states = [[0 for _ in range(len(text[0]))] for _ in range(len(text))]
        
    # wypełniam macierz stanów dla kolejnych kolumn
    for j in range(len(text[0])):
        # kolumna j
        node = automata
        for i in range(len(text)):
            while node != None and not text[i][j] in node.goto:
                node = node.fail
            if node == None:
                node = automata
                continue
            node = node.goto[text[i][j]]
            states[i][j] = node.index
    
    time2 = time()
    # dla każdego wiersza macierzy stanów poszukuje "patterns_pattern"
    i = -1 * len(patterns[0]) + 1
    count = 0
    max_time = 0
    for t in range(threads):
        time3 = time()
        for row in states[t*ceil(len(states)/threads):(t+1)*ceil(len(states)/threads)]:
            lenS, S = KMPSearch(row,patterns_pattern)
            count += lenS
            for j in S:
                print("Pattern found: (" + str(i) + "," + str(j) + ") ... (" + str(i) + "," + str(j+len(patterns)-1) + ")")
                print("               ...")
                print("               (" + str(i+len(patterns[0])-1) + "," + str(j) + ") ... (" + \
                      str(i+len(patterns[0])-1) + "," + str(j+len(patterns)-1) + ")")
            i += 1
        time4 = time()
        if (time4-time3 > max_time):
            max_time = time4-time3
            
    print("Total number of pattern occurences = " + str(count))    
    print("Automata building time => " + str(1000 * (time2-time1)) + " milliseconds")
    print("Searching time (longest thread time) => " + str(1000 * max_time) + " milliseconds")
    return count

In [41]:
count = search_patterns_threads(haystack_text,pat5,2)

Total number of pattern occurences = 0
Automata building time => 1920.8886623382568 milliseconds
Searching time (longest thread time) => 973.3965396881104 milliseconds


In [42]:
count = search_patterns_threads(haystack_text,pat5,4)

Total number of pattern occurences = 0
Automata building time => 1987.6835346221924 milliseconds
Searching time (longest thread time) => 507.64942169189453 milliseconds


In [43]:
count = search_patterns_threads(haystack_text,pat5,8)

Total number of pattern occurences = 0
Automata building time => 1936.8455410003662 milliseconds
Searching time (longest thread time) => 246.31500244140625 milliseconds


Jak widać wszystko działa zgodnie z założeniem -> czas wyszukiwania zmniejsza się około 2 razy wraz z dwukrotnym zwiększeniem ilości wątków.

# Wniosek

Zadanie pozwoliło zapoznać się z mechanizmem wyszukiwania wzorców dwuwymiarowym, problemem, który jest mocno rozchwytywany i wykorzystywany w wielu dziedzinach. Zastosowany algorytm można wykorzystać zarówno do plików tekstowych jak i graficznych (każdego pliku, który możemy w jakiś sposób zrzutować np. na macierz). Ostatnie polecenie pokazało, że łatwo zrównoleglić obliczenia (tutaj wyszukiwanie wzorca jednowymiarowego). Można również zrównoleglić wypełnianie macierzy stanów, ponieważ wyniki dla każdej kolumny są od siebie niezależne.